## ETL Yelp datasets

In [1]:
import pickle
import io
import pandas as pd
import json
from datetime import datetime
import os
import ast
from textblob import TextBlob

# Business

In [18]:
# Leer Archivo business.pkl
df_business = pd.read_pickle('datasets/Raw/business.pkl')

In [154]:
# Crear un contador para el sufijo
suffix_counter = {}

# Renombrar las columnas agregando sufijos
new_columns = []
for col in df_business.columns:
    if col in suffix_counter:
        # Columna duplicada: agregar sufijo '_duplicada'
        new_col = col + '_duplicada'
    else:
        # Columna no duplicada: agregar sufijo '_1' y actualizar contador
        suffix_counter[col] = 1
        new_col = col + ''
    new_columns.append(new_col)

# Asignar los nuevos nombres de columna al DataFrame
df_business.columns = new_columns

In [155]:
# Filtrar las columnas que contienen la palabra "duplicada" en su nombre
columnas_a_eliminar = [col for col in df_business.columns if 'duplicada' in col]

# Eliminar las columnas seleccionadas del DataFrame
df_business = df_business.drop(columns=columnas_a_eliminar)

In [156]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [157]:
duplicates_business = df_business.duplicated('business_id')
print(duplicates_business.sum())

0


In [158]:
# Estados contenidos en 'state'
print(df_business['state'].unique())

[nan 'CA' 'MO' 'AZ' 'PA' 'TN' 'FL' 'IN' 'LA' 'AB' 'NV' 'ID' 'DE' 'IL' 'NJ'
 'NC' 'CO' 'WA' 'HI' 'UT' 'TX' 'MT' 'MI' 'SD' 'XMS' 'MA' 'VI' 'VT']


In [159]:
# Filtar registros de Florida
df_business = df_business.loc[df_business['state'] == 'FL']

In [160]:
df_business.isna().sum()

business_id        0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes      2446
categories        18
hours           4108
dtype: int64

In [161]:
# Lista de columnas permitidas en el orden deseado
columnas_permitidas = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories', 'hours']

# Filtrar las columnas que se van a eliminar
columnas_a_eliminar = [col for col in df_business.columns if col not in columnas_permitidas]

# Imprimir las columnas que se eliminarán
print("Columnas a eliminar:")
for col in columnas_a_eliminar:
    print(col)

# Filtrar las columnas permitidas
df_business = df_business[columnas_permitidas]

Columnas a eliminar:
is_open
attributes


In [162]:
# Renombrar columna 'category' a 'tags'
df_business.rename(columns={'categories': 'tags'}, inplace=True)

In [163]:
# Convertir valores a float
df_business['latitude'] = df_business['latitude'].astype(float)
df_business['longitude'] = df_business['longitude'].astype(float)
df_business['stars'] = df_business['stars'].astype(float)

In [164]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26329 entries, 10 to 150320
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   26329 non-null  object 
 1   name          26329 non-null  object 
 2   address       26329 non-null  object 
 3   city          26329 non-null  object 
 4   state         26329 non-null  object 
 5   postal_code   26329 non-null  object 
 6   latitude      26329 non-null  float64
 7   longitude     26329 non-null  float64
 8   stars         26329 non-null  float64
 9   review_count  26329 non-null  object 
 10  tags          26311 non-null  object 
 11  hours         22221 non-null  object 
dtypes: float64(3), object(9)
memory usage: 2.6+ MB


In [165]:
# Eliminar los registros con valores nulos en la columna 'tags'
df_business.dropna(subset=['tags'], inplace=True)

#### Filtros

In [166]:
# Lista de palabras clave gatronomia
gastronomia = ['Restaurant', 'Restaurants','Food','Bars','Bar','Café','Coffeehouse','Bistro','Tavern','Buffet','Brewpub','Pub','Brasserie','Specialty Coffee Shop','Pub','Churrería','Diner','Dining','Teahouse','Tea Room','Gas Station', 'Gas','Fuel Station','Fuel']

# Filtrar los registros que contienen al menos una palabra clave en 'tags'
df_business_gastronomia = df_business[df_business['tags'].str.contains('|'.join(gastronomia))]

# Lista de palabras clave comida ethnica
comida_ethnica = ['Chinese','Indian','Thai','Italian','Greek','Helthy','Helth','Latin','Mexican','Tacos','Burritos','Enchiladas','Argentinian','Vegan','Vegetarian','Peruvian','Ceviche','Lomo','Pisco','Colombian','Empanadas','Arepas','Asian','Japanese','Sushi','Ramen','Sashimi','Tempura','Korean','Kimchi','Vietnamese','African','Ethiopian','Nigerian','Middle Eastern','Lebanese','Hummus','Falafel','Shawarma','Tabbouleh','Israeli','Shakshuka','Falafel','Hummus','Iranian','Healthy','Vegetarian','Vegan','Gas Station', 'Gas','Fuel Station','Fuel']

# Filtrar los registros que contienen al menos una palabra clave en 'tags'
df_business_ethnica = df_business_gastronomia[df_business_gastronomia['tags'].str.contains('|'.join(comida_ethnica))]

In [167]:
# Filtrar los registros donde 'name' se repite más de una vez
# Estos registros se consideraran como franquicias al identificarse con el mismo nombre y encontrarse en distintas locaciones
df_business = df_business_ethnica[df_business_ethnica.duplicated(subset='name', keep=False)]
df_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,tags,hours
82,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,FL,19104,39.954573,-75.194894,3.0,56,"Restaurants, Automotive, Delis, Gas Stations, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
200,vPcfJ3rm3NpdqVDod2WxLA,China Pearl,2E Summit Sq Shopping Ctr,Langhorne,FL,19047,40.217784,-74.926586,1.5,27,"Japanese, Chinese, Restaurants, Thai","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
348,OfW-VK04qr7b7bPm5QbyLg,Taqueria Feliz,303 Horsham Rd,Horsham,FL,19044,40.180727,-75.134699,3.0,77,"Mexican, Pizza, Restaurants","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:..."


In [168]:
# Función para encontrar palabras clave en la columna 'tags' y simplificar clasificacion restaurantes

def encontrar_palabra_clave(tags):

    # Lista de palabras clave que determian categorias mas abarcativas/comunes
    palabras_clave = ['Chinese','Indian','Thai','Italian','Greek','Mexican','Argentinian','Peruvian','Colombian','Japanese','Korean','Vietnamese','Ethiopian','Nigerian','Lebanese','Israeli','Iranian','Latin','Asian','African','Middle Eastern','Healthy','Vegetarian','Vegan','Casual','Gas Station']
    # Diccionario para determinar categoria segun palabras claves secundarias
    sub_tags = {  'Mexican' : ['Tacos','Burritos','Enchiladas'],
                        'Perubian' : ['Ceviche','Pisco'],
                        'Argentinian' : ['Asado','Locro','Dulce de Leche','Choripan'],
                        'Colombian' : ['Arepas'],
                        'Latin' : ['Empanadas','Lomo'],
                        'Asian' : ['Sushi','Ramen','Pan-Asian'],
                        'Korean' : ['Kimchi'],
                        'Japanese' : ['Sashimi','Tempura'],
                        'Middle_astern' : ['Hummus','Falafel','Shawarma','Tabbouleh'],
                        'Israeli' : ['Shakshuka'],
                        'Gas Station' : ['Gas', 'Gas Station', 'Convenience Store', 'Atm', 'Coffee Shop','Fuel Supplier','Fuel']}


    # Buscar en palabras_clave
    for palabra in palabras_clave:
        if palabra in tags:
            return palabra
    
    # Buscar en sub_tags
    for clave, valor in sub_tags.items():
        if any(sub_categoria in tags for sub_categoria in valor):
            return clave
        
    # Para aquellos registros donde se etiquete como desconocido, revisar columna tags para agregar etiquetas al diccionario sub_categories y poder reclasificarlo
    return 'Desconocido'


In [ ]:
# Encontrar el índice de la columna 'tags'
tags_index = df_business.columns.get_loc('tags')

# Insertar la nueva columna 'categories' antes de 'tags'
df_business.insert(tags_index - 1, 'categories', df_business['tags'].apply(encontrar_palabra_clave))

# Completar la columna categories con la palabra clave encontrada
df_business.loc[:, 'categories'] = df_business['tags'].apply(encontrar_palabra_clave)

In [144]:
df_business.reset_index(drop=True, inplace=True)

In [170]:
# Exportar business dataset como csv
df_business.to_csv('datasets/procesados/Yelp_business_procesado.csv', index=False)

# Check-in

In [171]:
# Path a checkin.json
file_path = os.path.join('datasets/Raw/checkin.json')

# Inicializar lista vacia check_in_row
check_in_row = []

# Abrir archivo JSON comprimido en read-text mode with UTF-8 encoding
with open(file_path, 'rt', encoding='utf-8') as file:
    # Iterate over each line in the file
    for line in file.readlines():
        # Usar ast.literal_eval para convertir cada linea JSON a Python dict
        check_in = ast.literal_eval(line)
        check_in_row.append(check_in)

# Crear rDataFrame con lista de diccinarios
df_check_in = pd.DataFrame(check_in_row)

In [172]:
df_check_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [173]:
# Filtrar df_check_in basado en los business_id de df_business
df_check_in = df_check_in[df_check_in['business_id'].isin(df_business['business_id'])]

In [174]:
# Función para normalizar una fecha individual en una cadena con múltiples fechas
def normalize_dates(date_str):
    dates_list = date_str.split(", ")
    return [datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in dates_list]

# Normalizar la columna 'date' que contiene múltiples fechas en una sola cadena
df_check_in['date'] = df_check_in['date'].apply(normalize_dates)

# Explode la lista de fechas normalizadas en múltiples filas
df_check_in = df_check_in.explode('date')

In [175]:
# Renombrar la columna 'date' a 'date_and_hour'
df_check_in.rename(columns={'date': 'date_and_hour'}, inplace=True)

# Convertir la columna 'date_and_hour' a formato de cadena de texto
df_check_in['date_and_hour'] = df_check_in['date_and_hour'].astype(str)

# Dividir la columna 'date_and_hour' en dos nuevas columnas: 'date' y 'hour'
df_check_in[['date', 'hour']] = df_check_in['date_and_hour'].str.split(' ', expand=True)

# Convertir las columnas 'date' y 'hour' al formato datetime
df_check_in['date'] = pd.to_datetime(df_check_in['date'])
df_check_in['hour'] = pd.to_datetime(df_check_in['hour'], format='%H:%M:%S').dt.time

# Eliminar la columna 'date_and_hour'
df_check_in.drop(columns=['date_and_hour'], inplace=True)

In [176]:
# Filtrar las filas a partir del 2021 
df_check_in = df_check_in[df_check_in['date'] >= '2020-01-01'].reset_index(drop=True)
df_check_in.head()

,business_id,date,hour
0,-K_FDDqmOt6hJLTOPgpnlA,2021-04-04,00:04:53
1,-K_FDDqmOt6hJLTOPgpnlA,2021-06-13,23:23:27
2,-K_FDDqmOt6hJLTOPgpnlA,2021-07-19,00:28:53
3,-Pq_I1Dqc7W0xf3A6SJTpQ,2021-01-18,19:31:49
4,-Pq_I1Dqc7W0xf3A6SJTpQ,2021-05-31,17:26:53


In [177]:
df_check_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2759 entries, 0 to 2758
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   business_id  2759 non-null   object        
 1   date         2759 non-null   datetime64[ns]
 2   hour         2759 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 64.8+ KB


In [178]:
df_check_in.reset_index(drop=True, inplace=True)

In [179]:
# Exportacion de df_check_in como csv
df_check_in.to_csv('datasets/procesados/Yelp_check_in_procesado.csv', index=False)

# Review

In [4]:
# Path review.jason
file_path = os.path.join('datasets/Raw/review.json')

# lista para almacenar reviews
review_row = []

# Filtro de date
date_threshold = '2021-00-00 00:00:00'

# Abrir archivo JSON comprimido en read-text mode con UTF-8 encoding
with open(file_path, 'rt', encoding='utf-8') as file:
    # Iterar sobre cada fila del archivo
    for line in file.readlines():
        # Usar ast.literal_eval para convertit cada linea JSON a Python dict
        review = ast.literal_eval(line)
        # Aplicar fitros
        if review['date'] >= date_threshold:
            review_row.append(review)

# Crear dataframe con lista de diccionarios
df_review = pd.DataFrame(review_row)

In [3]:
df_business = pd.read_csv('datasets/procesados/Yelp_business_procesado.csv')

In [6]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649854 entries, 0 to 649853
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_id    649854 non-null  object 
 1   user_id      649854 non-null  object 
 2   business_id  649854 non-null  object 
 3   stars        649854 non-null  float64
 4   useful       649854 non-null  int64  
 5   funny        649854 non-null  int64  
 6   cool         649854 non-null  int64  
 7   text         649854 non-null  object 
 8   date         649854 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 44.6+ MB


In [7]:
# Filtrar df_review basado en los business_id de df_business
df_review = df_review[df_review['business_id'].isin(df_business['business_id'])]

In [8]:
# Renombrar la columna 'date' a 'date_and_hour'
df_review.rename(columns={'date': 'date_and_hour'}, inplace=True)

# Convertir la columna 'date_and_hour' a formato de cadena de texto
df_review['date_and_hour'] = df_review['date_and_hour'].astype(str)

# Dividir la columna 'date_and_hour' en dos nuevas columnas: 'date' y 'hour'
df_review[['date', 'hour']] = df_review['date_and_hour'].str.split(' ', expand=True)

# Convertir las columnas 'date' y 'hour' al formato datetime
df_review['date'] = pd.to_datetime(df_review['date'])
df_review['hour'] = pd.to_datetime(df_review['hour'], format='%H:%M:%S').dt.time

# Eliminar la columna 'date_and_hour'
df_review.drop(columns=['date_and_hour'], inplace=True)

In [9]:
# Filtrar las filas a partir del 2021
df_review = df_review[df_review['date'] >= '2021-01-01'].reset_index(drop=True)

In [10]:
# Definir función para obtener la puntuación de sentimiento
def get_sentiment_score(text):
    if pd.isnull(text) or text == "":
        return 1  # Valor neutral si el texto está vacío o es NaN
    elif isinstance(text, str):
        # Crear un objeto TextBlob
        blob = TextBlob(text)

        # Obtener la puntuación de sentimiento
        sentiment_score = blob.sentiment.polarity

        # Escalar la puntuación entre 1 y 5
        scaled_score = int(round(5 * sentiment_score))

        # Asignar la puntuación
        if scaled_score <= 2:
            return 1
        elif scaled_score <= 3:
            return 2
        elif scaled_score <= 4:
            return 3
        else:
            return 4
    else:
        return 1  # Valor neutral para datos que no son de tipo cadena

In [11]:

# Asegurarse de que la columna 'text' sea de tipo cadena
df_review["text"] = df_review["text"].astype(str)

# Aplicar la función de análisis de sentimiento a la columna 'text' y crear una nueva columna 'sentiment_analysis'
df_review["sentiment_analysis"] = df_review["text"].apply(get_sentiment_score)

# Convertir la columna 'sentiment_analysis' al tipo de dato flotante si es necesario
df_review["sentiment_analysis"] = df_review["sentiment_analysis"].astype(int)

# Elimnar columna 'text'
df_review.drop(["text"], axis=1, inplace=True)

In [12]:
# Lista de columnas permitidas en el orden deseado
columnas_permitidas = ['review_id', 'user_id', 'business_id', 'stars', 'date', 'hour', 'sentiment_analysis']

# Filtrar las columnas que se van a eliminar
columnas_a_eliminar = [col for col in df_review.columns if col not in columnas_permitidas]

# Imprimir las columnas que se eliminarán
print("Columnas a eliminar:")
for col in columnas_a_eliminar:
    print(col)

# Filtrar las columnas permitidas
df_review = df_review[columnas_permitidas]

Columnas a eliminar:
useful
funny
cool


In [13]:
# Verificar la información del DataFrame df_review
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3689 entries, 0 to 3688
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   review_id           3689 non-null   object        
 1   user_id             3689 non-null   object        
 2   business_id         3689 non-null   object        
 3   stars               3689 non-null   float64       
 4   date                3689 non-null   datetime64[ns]
 5   hour                3689 non-null   object        
 6   sentiment_analysis  3689 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(4)
memory usage: 187.5+ KB


In [14]:
df_review.reset_index(drop=True, inplace=True)

In [15]:
# Exportar business dataset como csv
df_review.to_csv('datasets/procesados/Yelp_review_procesado.csv', index=False)